In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_excel("/content/drive/MyDrive/MTP phase 2/Type of work_excel.xlsx")
df.head(10)

,case no,text,class
0,21.0,This appeal is directed against the final judg...,0.0
1,21.0,Respondent No. 1-the Contractee Company was aw...,1.0
2,21.0,The total contract value as per the Letter of ...,0.0
3,21.0,"On 21.09.2012, the Contractee Company submitte...",0.0
4,22.0,A petition under Section 34 of the Arbitration...,0.0
5,22.0,At the outset it is observed that though the p...,0.0
6,22.0,The arbitral award emanates from a dispute bet...,1.0
7,22.0,The arbitral award has been passed by the arbi...,1.0
8,27.0,Both the cases are taken up together as in bot...,0.0
9,27.0,The present petition has been filed against th...,0.0


In [6]:
df = df.drop(['case no'], axis=1)

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
from nltk.corpus import stopwords

In [9]:
#Except = ['no']
def remove_stopwords(text):
  new_text = []
  for word in text.split():
    if word in stopwords.words('english'):# and word not in Except:
      new_text.append('')
    else:
      new_text.append(word)
  x = new_text[:]
  new_text.clear()
  return " ".join(x)

In [11]:
new_text = remove_stopwords(df['text'][0])

In [13]:
new_text

'This appeal  directed   final judgment  order dated 12.01.2015 passed   High Court  Karnataka  Bengaluru  C.M.P. No. 35  2014 whereby learned single Judge   High Court allowed  petition filed   respondent No. 1- Company  appointment   arbitrator  resolution   dispute   appellant-Company'

In [14]:
df['text'] = df['text'].apply(remove_stopwords)

In [15]:
df

,text,class
0,This appeal directed final judgment order ...,0.0
1,Respondent No. 1-the Contractee Company award...,1.0
2,The total contract value per Letter Accepta...,0.0
3,"On 21.09.2012, Contractee Company submitted ...",0.0
4,A petition Section 34 Arbitration & Concili...,0.0
...,...,...
95,The project inordinately delayed respondent...,0.0
96,"In 2004, bids invited, appellant, developm...",1.0
97,"The respondent bid, awarded contract, app...",0.0
98,"On 24th July 1997, agreement entered into, ...",1.0


In [16]:
import string

In [17]:
def text_cleaning(a):
  remove_punctuation = [char for char in a if char not in string.punctuation]
  remove_punctuation = ''.join(remove_punctuation)
  return [word for word in remove_punctuation.split()]

In [18]:
print(df.iloc[:,0].apply(text_cleaning))

0     [This, appeal, directed, final, judgment, orde...
1     [Respondent, No, 1the, Contractee, Company, aw...
2     [The, total, contract, value, per, Letter, Acc...
3     [On, 21092012, Contractee, Company, submitted,...
4     [A, petition, Section, 34, Arbitration, Concil...
                            ...                        
95    [The, project, inordinately, delayed, responde...
96    [In, 2004, bids, invited, appellant, developme...
97    [The, respondent, bid, awarded, contract, appe...
98    [On, 24th, July, 1997, agreement, entered, int...
99    [The, total, value, contract, Rs1607905450whic...
Name: text, Length: 100, dtype: object


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer = text_cleaning).fit(df['text'])

In [21]:
title_bow = bow_transformer.transform(df['text'])

In [22]:
x = title_bow.toarray()

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(title_bow)
title_tfidf = tfidf_transformer.transform(title_bow)

In [24]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(title_tfidf, df['class'], test_size = 0.2, random_state = 0)

In [27]:
model1 = MultinomialNB().fit(X_train, y_train)
model2 = BernoulliNB().fit(X_train, y_train)
model3 = LinearSVC().fit(X_train, y_train)
model4 = LogisticRegression().fit(X_train, y_train)
model5 = KNeighborsClassifier().fit(X_train, y_train)
model6 = DecisionTreeClassifier().fit(X_train, y_train)

In [28]:
all_pred1 = model1.predict(X_test)
all_pred2 = model2.predict(X_test)
all_pred3 = model3.predict(X_test)
all_pred4 = model4.predict(X_test)
all_pred5 = model5.predict(X_test)
all_pred6 = model6.predict(X_test)

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, all_pred3)

array([[12,  3],
       [ 1,  4]])

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
print('MultinomialNB:',accuracy_score(y_test,all_pred1))
print('BernoulliNB:',accuracy_score(y_test,all_pred2))
print('LinearSVC:',accuracy_score(y_test,all_pred3))
print('LogisticRegression:',accuracy_score(y_test,all_pred4))
print('KNeighborsClassifier:',accuracy_score(y_test,all_pred5))
print('DecisionTreeClassifier:',accuracy_score(y_test,all_pred6))

MultinomialNB: 0.9
BernoulliNB: 0.9
LinearSVC: 0.8
LogisticRegression: 0.9
KNeighborsClassifier: 0.85
DecisionTreeClassifier: 0.8
